## Generating Animated Plots with Pandas_Alive

In this tutorial we'll learn how to create a series of animations using Pandas_Alive. This post is rendered in the style of a Jupyter Notebook. Find the source here: [INSERT BASE LINK]

In [1]:
import pandas_alive

In [2]:
urban_df = pandas_alive.load_dataset("urban_pop")
display(urban_df.head())

,United States,India,China,Ethiopia,Poland,Malaysia,Peru,Venezuela,Iraq,Saudi Arabia,...,Iran,Turkey,Germany,Pakistan,Nigeria,Mexico,Russia,Japan,Indonesia,Brazil
year,,,,,,,,,,,,,,,,,,,,,
1976-01-01,160611122,138219074,162497601,3194879,19215135,4802814,9834645,10382196,7530612,4673719,...,15742290,16914583,56885943,18365687,13177979,38883279,90810675,85642808,26596457,67790415
1977-01-01,162256678,143699557,165293316,3300643,19625330,5038232,10197056,10778965,7901451,5041250,...,16521005,17474315,56801863,19175026,13868037,40371908,92479950,86538157,28001978,70478354
1978-01-01,164005080,149379782,171153535,3406129,20007316,5284698,10567468,11185526,8253727,5441027,...,17336704,18048189,56796181,20032626,14598364,41879098,94157479,87391419,29468690,73245834
1979-01-01,165847531,155285824,180399661,3522584,20341874,5539880,10945723,11600322,8598800,5885282,...,18227951,18640066,56865826,20941767,15355488,43407693,95659612,88197927,30996679,76091693
1980-01-01,167551171,161444128,189947471,3658252,20663601,5801267,11331194,12022351,8945814,6382806,...,19206467,19252680,57028530,21906732,16131172,44952217,96960865,88958689,32591870,79015954


In [5]:
urban_df.plot_animated(filename='urban-population-bar-chart-race.gif')

Generating BarChartRace, plotting ['United States', 'India', 'China', 'Ethiopia', 'Poland', 'Malaysia', 'Peru', 'Venezuela', 'Iraq', 'Saudi Arabia', 'Canada', 'Algeria', 'Ukraine', 'Vietnam', 'Thailand', 'Congo, Dem. Rep.', 'Spain', 'South Africa', 'Colombia', 'Argentina', 'Egypt', 'South Korea', 'Italy', 'Philippines', 'France', 'United Kingdom', 'Bangladesh', 'Iran', 'Turkey', 'Germany', 'Pakistan', 'Nigeria', 'Mexico', 'Russia', 'Japan', 'Indonesia', 'Brazil']


KeyboardInterrupt: 